## Extract all fields and check which aren't mapped

### Usage
Run All

Output will give two lists for each form:
- missingV1Mappings
    - Fields in the v1 form which do not have a mapping in v2    
- extraV2Mappings
    - Fields in the v2 form which do not exist in v1

Any country specific mappings can be added to `helpers/countryMappings.ts`



In [ ]:
import { authenticate } from './authentication.ts';
import { GATEWAY } from './helpers/routes.ts';

const token = await authenticate(GATEWAY, 'k.mweene', 'test');

In [ ]:
import { fetchEvents, fetchForms } from "./helpers/formsHandlers.ts";

const forms = await fetchForms(token)
const events = await fetchEvents(token)

In [ ]:
import { extractFieldType, extractFormFields } from './helpers/utils.ts'

const birthFormFields = extractFormFields(forms, 'birth').map((f) => f.id)
const deathFormFields = extractFormFields(forms, 'death').map((f) => f.id)

const birthEvent = events.find((x) => x.id === 'v2.birth')
const deathEvent = events.find((x) => x.id === 'v2.death')

const birthEventFields = new Set(
  extractFieldType(birthEvent, 'fields')
    .map((f) => f.id)
    .filter((x) => x)
)

const deathEventFields = new Set(
  extractFieldType(deathEvent, 'fields')
    .map((f) => f.id)
    .filter((x) => x)
)

birthEventFields


In [ ]:
import {
  DEFAULT_FIELD_MAPPINGS,
  MAPPING_FOR_CUSTOM_FIELDS,
} from './helpers/defaultMappings.ts'
import { COUNTRY_FIELD_MAPPINGS } from './helpers/countryMappings.ts'

const allMappings = {
  ...DEFAULT_FIELD_MAPPINGS,
  ...MAPPING_FOR_CUSTOM_FIELDS,
  ...COUNTRY_FIELD_MAPPINGS,
}

function partition<T>(list: T[], predicate: (item: T) => boolean) {
  const mapped: T[] = []
  const unmapped: T[] = []

  for (const item of list) {
    ;(predicate(item) ? mapped : unmapped).push(item)
  }

  return { mapped, unmapped }
}

const birthFormMappings = partition(birthFormFields, (f) => allMappings[f])
const deathFormMappings = partition(deathFormFields, (f) => allMappings[f])

const usedBirthMappings = Object.fromEntries(
  birthFormMappings.mapped.map((field) => [field, allMappings[field]])
)
const usedDeathMappings = Object.fromEntries(
  deathFormMappings.mapped.map((field) => [field, allMappings[field]])
)

const birthEventMappings = partition(birthEventFields, (f) =>
  Object.values(usedBirthMappings).includes(f)
)
const deathEventMappings = partition(deathEventFields, (f) =>
  Object.values(usedDeathMappings).includes(f)
)

const unmappedFields = {
  birth: {
    missingV1Mappings: birthFormMappings.unmapped,
    extraV2Mappings: birthEventMappings.unmapped,
  },
  death: {
    missingV1Mappings: deathFormMappings.unmapped,
    extraV2Mappings: deathEventMappings.unmapped,
  },
}
unmappedFields
